## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-10-02-29-17 +0000,nypost,Secret Service suspends 6 agents assigned to p...,https://nypost.com/2025/07/09/us-news/secret-s...
1,2025-07-10-02-27-12 +0000,nypost,"Illinois father, 65, and his autistic son, 20,...",https://nypost.com/2025/07/09/us-news/illinois...
2,2025-07-10-02-22-47 +0000,nypost,Mafia captain from one of NYC’s ‘five families...,https://nypost.com/2025/07/09/us-news/lucchese...
3,2025-07-10-02-18-55 +0000,nypost,Tatted-up Florida man accused of threatening e...,https://nypost.com/2025/07/09/us-news/florida-...
4,2025-07-10-02-01-26 +0000,bbc,Kellogg shares soar on reports of Ferrero take...,https://www.bbc.com/news/articles/c3d1m2kjk93o


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2341/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
6,trump,24
40,he,7
153,israel,6
250,new,6
59,she,5


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
67,2025-07-09-16-39-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,55
108,2025-07-09-03-50-00 +0000,wsj,President Trump decided to delay the implement...,https://www.wsj.com/economy/trade/trump-tariff...,53
9,2025-07-10-01-17-00 +0000,wsj,A brewing debate inside the Federal Reserve ov...,https://www.wsj.com/economy/central-banking/wh...,52
42,2025-07-09-20-49-26 +0000,nypost,Trump dismisses Iran threatening poolside Mar-...,https://nypost.com/2025/07/09/us-news/trump-sa...,49
84,2025-07-09-12-38-45 +0000,nypost,Trump boasted to donors that he threatened to ...,https://nypost.com/2025/07/09/us-news/trump-bo...,49


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
67,55,2025-07-09-16-39-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
32,31,2025-07-09-22-38-58 +0000,nypost,Livvy Dunne struck out in bid to buy Babe Ruth...,https://nypost.com/2025/07/09/sports/livvy-dun...
3,29,2025-07-10-02-18-55 +0000,nypost,Tatted-up Florida man accused of threatening e...,https://nypost.com/2025/07/09/us-news/florida-...
50,28,2025-07-09-19-48-23 +0000,nyt,Former President Yoon Suk Yeol of South Korea ...,https://www.nytimes.com/2025/07/09/world/asia/...
36,23,2025-07-09-22-02-53 +0000,nypost,North Carolina facing another flood threat jus...,https://nypost.com/2025/07/09/us-news/north-ca...
93,23,2025-07-09-10-00-00 +0000,nypost,Secretary of State Marco Rubio gets glowing en...,https://nypost.com/2025/07/09/us-news/secretar...
18,21,2025-07-10-00-29-32 +0000,nypost,Hundreds of huge Amazon packages arrive at Cal...,https://nypost.com/2025/07/09/us-news/hundreds...
35,21,2025-07-09-22-02-56 +0000,nypost,"Gossipy small-town activist drops F-bombs, ins...",https://nypost.com/2025/07/09/us-news/gossipy-...
43,20,2025-07-09-20-37-43 +0000,nypost,Mike Johnson steps up calls for fresh Russia s...,https://nypost.com/2025/07/09/us-news/mike-joh...
108,19,2025-07-09-03-50-00 +0000,wsj,President Trump decided to delay the implement...,https://www.wsj.com/economy/trade/trump-tariff...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
